## RAG Application using typesense

In [2]:
import typesense
import os
from dotenv import load_dotenv
load_dotenv()

python-dotenv could not parse statement starting at line 5


True

In [ ]:
client = typesense.Client({
    'nodes': [{
        'host': 'cfa0h82do73y6um9p-1.a1.typesense.net',
        'port': '443',
        'protocol': 'https'
    }],
    'api_key': 'WJnmvDQwVYpRRhe9Ur7J9ffZQEhO8Aer',
    'connection_timeout_seconds': 10
})

books_schema = {
    'name': 'books',
    'fields': [
        {'name': 'title', 'type': 'string'},
        {'name': 'authors', 'type': 'string[]', 'facet': True},
        {'name': 'publication_year', 'type': 'int32', 'facet': True},
        {'name': 'ratings_count', 'type': 'int32'},
        {'name': 'average_rating', 'type': 'float'}
    ],
    'default_sorting_field': 'ratings_count'
}

print(client.collections.create(books_schema))

In [19]:
client, books_schema

(<typesense.client.Client at 0x1ed1f09d700>,
 {'name': 'books',
  'fields': [{'name': 'title', 'type': 'string'},
   {'name': 'authors', 'type': 'string[]', 'facet': True},
   {'name': 'publication_year', 'type': 'int32', 'facet': True},
   {'name': 'ratings_count', 'type': 'int32'},
   {'name': 'average_rating', 'type': 'float'}],
  'default_sorting_field': 'ratings_count'})

In [20]:
batch_size = 100
batch = []

with open('books.jsonl', 'r', encoding = 'utf-8') as f:
    data = f.read()
    client.collections['books'].documents.import_(data)

In [23]:
search_parameters = {
    'q': 'harry potter',
    'query_by': 'title, authors',
    'sort_by': 'ratings_count:desc',
    'filter_by': 'publication_year:<1998'
}

client.collections['books'].documents.search(search_parameters=search_parameters)

{'facet_counts': [],
 'found': 1,
 'hits': [{'document': {'authors': ['J.K. Rowling', ' Mary GrandPré'],
    'average_rating': 4.44,
    'id': '2',
    'image_url': 'https://images.gr-assets.com/books/1474154022m/3.jpg',
    'publication_year': 1997,
    'ratings_count': 4602479,
    'title': "Harry Potter and the Philosopher's Stone"},
   'highlight': {'title': {'matched_tokens': ['Harry', 'Potter'],
     'snippet': "<mark>Harry</mark> <mark>Potter</mark> and the Philosopher's Stone"}},
   'highlights': [{'field': 'title',
     'matched_tokens': ['Harry', 'Potter'],
     'snippet': "<mark>Harry</mark> <mark>Potter</mark> and the Philosopher's Stone"}],
   'text_match': 1157451471441100921,
   'text_match_info': {'best_field_score': '2211897868288',
    'best_field_weight': 15,
    'fields_matched': 1,
    'num_tokens_dropped': 0,
    'score': '1157451471441100921',
    'tokens_matched': 2,
    'typo_prefix_score': 0}}],
 'out_of': 9979,
 'page': 1,
 'request_params': {'collection_name

In [24]:
### Langchain + Typsense + Groq LLM + RAG Application

from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Typesense
from langchain_text_splitters import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq

e:\AgenticAI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
groq_api_key = os.getenv("GROQ_API_KEY")

In [29]:
loader = TextLoader('text.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
docs = text_splitter.split_documents(documents=documents)

embeddings = HuggingFaceEmbeddings()

C:\Users\GOURAV\AppData\Local\Temp\ipykernel_5424\512210283.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
C:\Users\GOURAV\AppData\Local\Temp\ipykernel_5424\512210283.py:6: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


In [30]:
docsearch = Typesense.from_documents(
    docs,
    embeddings,
    typesense_client_params = {
        'host' : 'cfa0h82do73y6um9p-1.a1.typesense.net',
        'protocol': 'https',
        'port': '443',
        'typesense_api_key': 'WJnmvDQwVYpRRhe9Ur7J9ffZQEhO8Aer',
        'typesense_collection_name': 'lang-chain'
    },
    
)

In [33]:
query = 'what is artificial intelligence'
found_docs = docsearch.similarity_search(query)
print(found_docs[0].page_content)

Artificial Intelligence

Artificial Intelligence (AI) is one of the most transformative technologies of the 21st century. It refers to the ability of machines and computer systems to perform tasks that usually require human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. AI has become an integral part of modern life, shaping industries, research, and even our daily activities.

At its core, AI is the simulation of human intelligence in machines. This involves developing algorithms and models that allow computers to analyze data, recognize patterns, and make decisions. AI can be broadly categorized into two types: Narrow AI and General AI. Narrow AI is designed for specific tasks, such as speech recognition or image classification, while General AI refers to machines that can perform any intellectual task that humans can do â€” a goal that still remains largely theoretical.


In [34]:
retriever = docsearch.as_retriever()
retriever

VectorStoreRetriever(tags=['Typesense', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.typesense.Typesense object at 0x000001ED6FA1F410>, search_kwargs={})

In [38]:
query = 'give the summary of the whole essay'
top_doc = retriever.invoke(query)[0]
print(top_doc.page_content)

In conclusion, Artificial Intelligence is no longer a futuristic concept but a present-day reality. While it offers immense opportunities for progress, it also demands careful management of its risks and ethical implications. The challenge for humanity is to balance innovation with responsibility, ensuring that AI serves as a tool to enhance human life rather than replace it.
